In [1]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
from tensorflow.keras.datasets import mnist

# Модель с последовательным проходом по слоям
from tensorflow.keras.models import Sequential

# Слои: полносвязный, Dropout, входной слой, вытягивающий в одномерный вектор 784х1
from tensorflow.keras.layers import Dense, Dropout, Flatten

# Различные виды вариантов расчета градиентного спуска
from tensorflow.keras.optimizers import SGD, RMSprop, Nadam

# Чтобы сделать выходы one-hot 
from tensorflow.keras import utils

# Инициализация параметров слоев
from tensorflow.keras import initializers

# Критерии качества
from tensorflow.keras.losses import categorical_crossentropy

# Не углублялся, но что-то вроде критерия останова
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# Нормализация к границам [0;1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [5]:
# One-hot encoding
Y_train = utils.to_categorical(y_train, 10)
Y_test = utils.to_categorical(y_test, 10)

In [ ]:
import keras_tuner as kt

In [ ]:
# Общее описание структуры нейросети
# Я использовал только 1 скрытый слой, теоретически можно через тюнер в цикле перебирать кол-во слоев
# также использовал только categorical_crossentropy в качестве критерия качества, можно перебирать и другие
def call_existing_code(units, activation, dropout, lr, do_rate, optimizer, nesterov, centered, mean, stddev, value):
    model = Sequential()
    
    init_weights = initializers.TruncatedNormal(mean=mean, stddev=stddev) # инициализация весов
    init_biases = initializers.Constant(value=value) # инициализация свободных членов
    
    model.add(Flatten()) # входной слой, "вытягивает" 28x28 в 784x1
    
    # Полносвязный слой с настройками, описанными в функции ниже
    model.add(Dense(units=units, activation=activation, kernel_initializer=init_weights, bias_initializer=init_biases))
    
    # Использовать ли dropout
    if dropout:
        model.add(Dropout(rate=do_rate))
        
    # Выходной слой на 10 выходов с softmax
    model.add(Dense(10, activation="softmax", kernel_initializer=init_weights, bias_initializer=init_biases))
    
    # Перебор различных оптимизаторов
    if optimizer == "Nadam":
        model.compile(
            optimizer=Nadam(learning_rate=lr),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
    elif optimizer == "SGD":
        model.compile(
            optimizer=SGD(learning_rate=lr, nesterov=nesterov),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
    elif optimizer == "RMSprop":
        model.compile(
            optimizer=RMSprop(learning_rate=lr, centered=centered),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
    return model


# Функция задания тюнером параметров для перебора
def model_builder(hp):
    
    # Кол-во нейронов, используется только в скрытом слое. Инициализируется как целое число, ранжирование задается с шагом
    units = hp.Int("units", min_value=128, max_value=512, step=128)
    
    # Выбор активационной функции скрытого слоя. Инициализируется как Enum (просто сравнение)
    activation = hp.Choice("activation", ["relu", "tanh"])
    
    # Логическая переменная, переберет всего лишь True и False, от которой будет зависеть, применять ли dropout
    dropout = hp.Boolean("dropout")
    
    # Перебор оптимизаторов
    optimizer = hp.Choice("optimizer", ["Nadam", "SGD", "RMSprop"])
    
    # Скорость обучения, используется во всех оптимизаторах. Инициализируется как число с плавающей запятой, перебор с логарифмическим шагом
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    # Процент неиспользуемых нейронов при dropout. Инициализируется как число с плавающей запятой, перебор с шагом
    do_rate = hp.Float("rate", min_value=2.5e-1, max_value=5e-1, step=2.5e-1)
    
    # Использовать ли поправку Нестерова при оптимизаторе SGD
    nesterov = hp.Boolean("nesterov")
    
    # Нормализовать ли градиенты (???) при оптимизаторе RMSprop
    centered = hp.Boolean("centered")
    
    # Среднее значение весов при инициализации
    mean = hp.Float("mean", min_value=0.0, max_value=1e-1, step=2.5e-2)
    
    # Среднеквадратическое отклонение весов при инициализации
    stddev = hp.Float("stddev", min_value=2.5e-1, max_value=1, step=2.5e-1)
    
    # Значение свободных членов при инициализации (вроде можно инициализировать тоже через распределение с mean и stddev)
    value = hp.Float("value", min_value=0, max_value=1e-2, step=5e-3)
    # Вызывает функцию call_existing_code, подставляя в нее параметры
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr, do_rate=do_rate, optimizer=optimizer, 
        nesterov=nesterov, centered=centered, mean=mean, stddev=stddev, value=value
    )
    return model

model_builder(kt.HyperParameters()) # Проверяет, скомпилится ли модель

In [ ]:
# Формирование тюнера, все стадии обучения будут сохраняться в папку directory
tuner = kt.Hyperband(model_builder, # Настроенная сетка параметров
                     objective='val_accuracy', # По каким данным смотреть улучшение модели
                     max_epochs=20,
                     directory='my_dir',
                     project_name='intro_to_kt',
                     overwrite=True
                    )

In [ ]:
# Критерий останова по валидационным данным, если точность на них ухудшается или не двигается patience раз
stop_early = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# ОБУЧЕНИЕ
tuner.search(X_train, Y_train, epochs=50, validation_split=0.2, callbacks=[TensorBoard("/my_dir/intro_to_kt/my_logs")])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /my_dir/intro_to_kt/my_logs

In [ ]:
# Получение наилучших параметров
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Лучшие параметры:
Кол-во нейронов скрытого слоя: {best_hps.get('units')}
Активационная функция скрытого слоя: {best_hps.get('activation')}
Использовать ли Dropout: {best_hps.get('dropout')}
Процент неиспользуемых нейронов при Dropout: {best_hps.get('rate')}
Оптимизатор: {best_hps.get('optimizer')}
Скорость обучения: {best_hps.get('lr')}
Использовать ли поправку Нестерова, если оптимизатор SGD: {best_hps.get('nesterov')}
Нормализовать ли градиенты, если оптимизатор RMSprop: {best_hps.get('centered')}
Среднее значение весов при инициализации: {best_hps.get('mean')}
Среднеквадратическое отклонение весов при инициализации: {best_hps.get('stddev')}
Значение свободных членов при инициализации: {best_hps.get('value')}
""")

In [ ]:
# Нахождение оптимального числа эпох
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))